# `pyproj` - преобразование координат и работа с системами координат в Python





**pyproj** — это `Python`-обёртка над библиотекой `PROJ`, которая является де-факто стандартом для работы с картографическими проекциями и трансформацией координат. `Pyproj` обеспечивает:

- Определение и анализ систем координат (`CRS`)
- Преобразования координат между любыми `CRS`
- Геодезические (geodesic) вычисления: расстояния, площади, азимуты
- Поддержку современных стандартов (`EPSG`, `WKT2`)
  
---

## 1. Установка и импорт

Установка через pip:
```bash
pip install pyproj
```
Импорт основных классов:
```python
import pyproj
from pyproj import CRS, Transformer, Geod
```


## 2. Системы координат (Coordinate Reference System, CRS)

### Что такое CRS?
**Система координат** — это математическая модель, задающая, как координаты точки соотносятся с физическим положением на Земле.

### Типы CRS в pyproj:
| Тип                | Описание                                                         | Пример          |
|--------------------|------------------------------------------------------------------|-----------------|
| Geographic (географическая)  | Широта/долгота на эллипсоиде                     | WGS84 (EPSG:4326) |
| Projected (проецированная)   | Плоские координаты (м), получены проекцией       | UTM (EPSG:32633)  |
| Vertical (вертикальная)      | Описание высотной системы                        | Baltic 1977       |
| Compound (составная)         | Горизонтальная + вертикальная CRS                | NAD83 + NAVD88    |

### Основные свойства CRS:
- **Датум (datum)**: геодезическая основа системы, определяет положение эллипсоида
- **Эллипсоид (ellipsoid)**: форма Земли (например, WGS84, GRS80, Красовского)
- **Проекция**: метод отображения на плоскость (Меркатор, UTM, Гаусса-Крюгера и др.)
- **Область действия (area_of_use)**: географические границы применимости

### Создание CRS в pyproj:
```python
from pyproj import CRS

crs = CRS.from_epsg(4326)           # По EPSG-коду
crs = CRS.from_proj4("+proj=longlat +datum=WGS84")  # По PROJ-строке
crs = CRS.from_wkt("...")           # По WKT-описанию
crs = CRS.from_authority("EPSG", 32633)  # По коду и организации
```


In [ ]:
from pyproj import CRS

# Создание CRS по EPSG-коду
crs_wgs84 = CRS.from_epsg(4326)
crs_utm33 = CRS.from_epsg(32633)

print("=== WGS84 ===")
print(f"Название: {crs_wgs84.name}")
print(f"Тип: географическая={crs_wgs84.is_geographic}, проецированная={crs_wgs84.is_projected}")
print(f"Датум: {crs_wgs84.datum}")
print(f"Эллипсоид: {crs_wgs84.ellipsoid}")
print()

print("=== UTM зона 33N ===")
print(f"Название: {crs_utm33.name}")
print(f"Тип: географическая={crs_utm33.is_geographic}, проецированная={crs_utm33.is_projected}")
print(f"Датум: {crs_utm33.datum}")
print(f"Область действия: {crs_utm33.area_of_use}")


### Примеры распространённых систем координат

| EPSG-код  | Название                        | Описание                                   |
|-----------|---------------------------------|-------------------------------------------|
| 4326      | WGS84                           | Географические координаты GPS (lat, lon)   |
| 3857      | Web Mercator (Pseudo-Mercator)  | Используется в веб-картах                  |
| 32633     | UTM зона 33N                    | Проекция для центральной Европы            |
| 4284      | Пулково-1942                    | Географическая CRS СССР                    |
| 28403     | Пулково-1942 / Гаусса-Крюгера 3 | Проекция в зоне 3 (СНГ)                    |
| 4269      | NAD83                           | Североамериканский датум 1983              |
| 2154      | RGF93 / Lambert-93              | Французская проекция                       |


## 3. Преобразование координат: класс Transformer

**Transformer** — основной объект для трансформации координат между CRS.

### Создание Transformer:
```python
from pyproj import Transformer

transformer = Transformer.from_crs(crs_source, crs_target)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32633")  # По строкам
```

### Ключевые параметры:
- `always_xy=True` — порядок координат всегда (X, Y), т.е. (lon, lat), а не (lat, lon)
- `area_of_interest` — ограничение области для выбора оптимальной трансформации

### Трансформация:
```python
x, y = transformer.transform(lat, lon)
```
Для массивов:
```python
xs, ys = transformer.transform([lat1, lat2], [lon1, lon2])
```


In [ ]:
from pyproj import CRS, Transformer

# Создаём CRS
crs_wgs84 = CRS.from_epsg(4326)
crs_utm33 = CRS.from_epsg(32633)

# Transformer: WGS84 -> UTM
transformer = Transformer.from_crs(crs_wgs84, crs_utm33)

# Преобразование одной точки (lat, lon)
lat, lon = 59.9386, 30.3141  # Санкт-Петербург
x, y = transformer.transform(lat, lon)
print(f"WGS84 ({lat}, {lon}) -> UTM33N: X={x:.2f}, Y={y:.2f}")

# Обратное преобразование
transformer_back = Transformer.from_crs(crs_utm33, crs_wgs84)
lat2, lon2 = transformer_back.transform(x, y)
print(f"UTM33N ({x:.2f}, {y:.2f}) -> WGS84: ({lat2:.5f}, {lon2:.5f})")

### Пакетное преобразование координат (batch transform)
Для преобразования множества точек передавайте списки координат:


In [ ]:
# Список координат: (lat, lon)
coords = [
    (59.9386, 30.3141),   # Санкт-Петербург
    (55.7512, 37.6184),   # Москва
    (56.8389, 60.6057),   # Екатеринбург
]

lats = [c[0] for c in coords]
lons = [c[1] for c in coords]

xs, ys = transformer.transform(lats, lons)

for (lat, lon), x, y in zip(coords, xs, ys):
    print(f"({lat}, {lon}) -> X={x:.2f}, Y={y:.2f}")

### Порядок осей и параметр `always_xy`
В некоторых CRS (например, EPSG:4326) порядок координат — (lat, lon), а не (lon, lat).
Чтобы всегда использовать порядок (lon, lat), используйте параметр `always_xy=True`:
```python
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32633", always_xy=True)
x, y = transformer.transform(lon, lat)
```


### Область интереса (Area of Interest)
При трансформации между CRS с разными датумами может быть несколько вариантов преобразования.
С помощью `AreaOfInterest` можно ограничить область для выбора наиболее точного преобразования.
```python
from pyproj.transformer import Transformer, AreaOfInterest
transformer = Transformer.from_crs(
    "EPSG:4326", "EPSG:2694",
    area_of_interest=AreaOfInterest(-136.46, 49.0, -60.72, 83.17)
)
```


### TransformerGroup: выбор из нескольких трансформаций
Если между CRS доступно несколько трансформаций, можно просмотреть их:
```python
from pyproj.transformer import TransformerGroup
tg = TransformerGroup("EPSG:4326", "EPSG:32633")
for t in tg.transformers:
    print(t)
```


## 4. Геодезические вычисления: класс Geod

**Geod** — для геодезических расчётов на эллипсоиде:
- Расстояние между точками
- Азимут (направление) между точками
- Вычисление координат по начальной точке, азимуту и расстоянию
- Площадь и периметр полигонов на сфероиде

### Создание:
```python
from pyproj import Geod
geod = Geod(ellps="WGS84")
```

### Основные методы:
| Метод            | Описание                                                      |
|------------------|---------------------------------------------------------------|
| `inv()`          | Расстояние и азимуты между двумя точками (lat, lon)           |
| `fwd()`          | Координаты точки по начальной точке, азимуту и расстоянию     |
| `line_length()`  | Длина линии по списку координат                               |
| `geometry_area_perimeter()` | Площадь и периметр полигона (Shapely-геометрия)   |
| `geometry_length()`         | Длина линии (Shapely-геометрия)                   |


In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

# Расстояние между Санкт-Петербургом и Москвой
lon1, lat1 = 30.3141, 59.9386  # Санкт-Петербург
lon2, lat2 = 37.6184, 55.7512  # Москва

az12, az21, dist = geod.inv(lon1, lat1, lon2, lat2)
print(f"Расстояние: {dist/1000:.2f} км")
print(f"Азимут прямой: {az12:.2f} градусов")
print(f"Азимут обратный: {az21:.2f} градусов")


In [ ]:
# Вычисление координат по начальной точке, азимуту и расстоянию (fwd)
lon_start, lat_start = 30.3141, 59.9386
azimuth = 135.0  # Направление на юго-восток
distance = 100000  # 100 км

lon_end, lat_end, back_az = geod.fwd(lon_start, lat_start, azimuth, distance)
print(f"Конечная точка: lon={lon_end:.5f}, lat={lat_end:.5f}")


In [ ]:
# Длина линии по списку координат
lons = [30.3141, 37.6184, 60.6057]
lats = [59.9386, 55.7512, 56.8389]
total_length = geod.line_length(lons, lats)
print(f"Общая длина маршрута: {total_length/1000:.2f} км")


### Площадь и периметр полигона
Для геометрий Shapely можно вычислить площадь и периметр по геодезической модели:
```python
from pyproj import Geod
from shapely.geometry import Polygon
geod = Geod(ellps="WGS84")
polygon = Polygon([(lon1, lat1), (lon2, lat2), ...])
area, perimeter = geod.geometry_area_perimeter(polygon)
```


In [ ]:
# Пример расчёта площади полигона
from pyproj import Geod
try:
    from shapely.geometry import Polygon
    geod = Geod(ellps="WGS84")
    # Координаты углов полигона (lon, lat)
    polygon = Polygon([
        (30.0, 59.0),
        (31.0, 59.0),
        (31.0, 60.0),
        (30.0, 60.0),
    ])
    area, perimeter = geod.geometry_area_perimeter(polygon)
    print(f"Площадь: {abs(area)/1e6:.2f} кв.км")
    print(f"Периметр: {perimeter/1000:.2f} км")
except ImportError:
    print("Для расчёта площади требуется библиотека shapely: pip install shapely")

## 5. PROJ-строки и WKT-описание

CRS можно задать или экспортировать в различных форматах:
- **PROJ-строка**: `+proj=utm +zone=33 +datum=WGS84`
- **WKT (Well-Known Text)**: текстовое описание CRS для обмена между системами

```python
crs = CRS.from_proj4("+proj=utm +zone=33 +datum=WGS84")
print(crs.to_proj4())
print(crs.to_wkt())
```


In [ ]:
from pyproj import CRS

crs = CRS.from_epsg(32633)
print("=== PROJ-строка ===")
print(crs.to_proj4())
print()
print("=== WKT (Well-Known Text) ===")
print(crs.to_wkt(pretty=True))

## 6. Работа с базой данных CRS

Модуль `pyproj.database` позволяет искать доступные CRS:
```python
from pyproj.database import query_crs_info
crs_list = query_crs_info(auth_name='EPSG', pj_types='PROJECTED_CRS')
```


In [ ]:
from pyproj.database import query_crs_info

# Получить первые 5 проецированных CRS из EPSG
crs_list = query_crs_info(auth_name='EPSG', pj_types='PROJECTED_CRS')
for info in crs_list[:5]:
    print(info)

## 7. Практический пример: GPS-координаты в местную систему

Типовая задача: преобразовать координаты из WGS84 (GPS) в местную проекцию (например, UTM или СК-42).


In [ ]:
from pyproj import CRS, Transformer

# Исходная система — WGS84
crs_gps = CRS.from_epsg(4326)

# Целевая система — Пулково 1942 / Гаусса-Крюгера зона 6
crs_sk42 = CRS.from_epsg(28406)

transformer = Transformer.from_crs(crs_gps, crs_sk42)

# GPS-координаты (lat, lon)
lat, lon = 55.7512, 37.6184  # Москва

x, y = transformer.transform(lat, lon)
print(f"GPS ({lat}, {lon}) -> СК-42 зона 6: X={x:.2f}, Y={y:.2f}")

## 8. Расширенные возможности: пайплайны (pipelines)

Для сложных преобразований можно создать pipeline — цепочку операций:
```python
pipeline = "+proj=pipeline +step +proj=longlat +datum=WGS84 +step +proj=utm +zone=33"
transformer = Transformer.from_pipeline(pipeline)
```
Пайплайны полезны, когда требуется нестандартная последовательность операций или явное задание параметров (towgs84, nadgrids и др.).


## 9. Типичные ошибки и рекомендации

| Ошибка                                       | Причина и решение                                                               |
|----------------------------------------------|---------------------------------------------------------------------------------|
| Неправильный порядок координат               | Используйте `always_xy=True` или проверяйте axis_info для CRS                   |
| Разные результаты на разных версиях          | Используйте TransformerGroup, явно указывайте трансформацию                     |
| Ошибка "Missing grid"                        | Установите дополнительные datum grids (см. документацию pyproj)                 |
| CRS не найден                                | Проверьте EPSG-код на epsg.io или используйте WKT/PROJ-строку                   |

---


## 10. Полезные ссылки

- [Официальная документация pyproj](https://pyproj4.github.io/pyproj/stable/)
- [EPSG.io — поиск систем координат](https://epsg.io/)
- [PROJ — библиотека преобразований](https://proj.org/)
- [spatialreference.org](https://spatialreference.org/)

---
**Этот конспект подготовлен для работы с pyproj >= 3.0**


---
---